In [1]:
import requests
import pandas as pd
import time
import numpy as np
from datetime import datetime

# Import the agent classes
from agent_ma import MovingAverageCrossoverAgent
from agent_dummy import DummyAgent
from agent_BollingerBands import BollingerBandsAgent
from agent_macd import MACDAgent
from agent_momentum import MomentumAgent
from agent_mr import MeanReversionAgent
from agent_rsi import RSIAgent
from agent_composite import CompositeAgent
from agent_emav import EMAVAgent


# Fetch live data from Binance API

In [2]:
def fetch_live_data(symbol='BTCUSDT', interval='1m'):
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit=1'
    response = requests.get(url)
    data = response.json()
    latest_data = data[0]
    return {
        'timestamp': pd.to_datetime(latest_data[0], unit='ms'),
        'Open': float(latest_data[1]),
        'High': float(latest_data[2]),
        'Low': float(latest_data[3]),
        'Close': float(latest_data[4]),
        'Volume': float(latest_data[5])
    }


# Fetch historical data from Binance API

In [3]:
def fetch_historical_data(symbol, interval, limit=1000):
    """
    interval:
    1m, 3m, 5m, 15m, 30m
    1h, 2h, 4h, 6h, 8h, 12h,
    1d, 3d, 1w, 1M
    """
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
    return df

# Preprocess data by adding returns

In [4]:
def preprocess_data(df):
    df['returns'] = df['Close'].pct_change()
    return df


# Function to update the DataFrame with new live data

In [5]:
# Function to update the DataFrame with new live data
def update_data(df):
    live_data = fetch_live_data()
    new_data = pd.DataFrame([live_data])
    new_data.set_index('timestamp', inplace=True)
    
    df = pd.concat([df, new_data])
    df = preprocess_data(df)
    return df


# Initialize

In [6]:
# Initialize DataFrame to store live data
df = pd.DataFrame(columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])

# Initialize agents
mac_agent = MovingAverageCrossoverAgent()
dummy_agent = DummyAgent()
mean_reversion_agent = MeanReversionAgent()
momentum_agent = MomentumAgent()
macd_agent = MACDAgent()
bollinger_bands_agent = BollingerBandsAgent()
rsi_agent = RSIAgent()
composite_agent = CompositeAgent()
emav_agent = EMAVAgent()

# Trade

In [7]:
# Function to update agents and print portfolio values
def update_agents(df, current_time, interval_name):
    df = update_data(df)
    current_price = df['Close'].iloc[-1]
    mac_agent.trade(df)
    dummy_agent.trade(df)
    mean_reversion_agent.trade(df)
    momentum_agent.trade(df)
    macd_agent.trade(df)
    bollinger_bands_agent.trade(df)
    rsi_agent.trade(df)
    composite_agent.trade(df)
    emav_agent.trade(df)
    print(f"{interval_name} - MAC Agent Portfolio Value: {mac_agent.get_portfolio_value(current_price)}")
    print(f"{interval_name} - Dummy Agent Portfolio Value: {dummy_agent.get_portfolio_value(current_price)}")
    print(f"{interval_name} - MeanReversion Agent Portfolio Value: {mean_reversion_agent.get_portfolio_value(current_price)}")
    print(f"{interval_name} - Momentum Agent Portfolio Value: {momentum_agent.get_portfolio_value(current_price)}")
    print(f"{interval_name} - MACD Agent Portfolio Value: {macd_agent.get_portfolio_value(current_price)}")
    print(f"{interval_name} - BollingerBands Agent Portfolio Value: {bollinger_bands_agent.get_portfolio_value(current_price)}")
    print(f"{interval_name} - RSI Agent Portfolio Value: {rsi_agent.get_portfolio_value(current_price)}")
    print(f"{interval_name} - Composite Agent Portfolio Value: {composite_agent.get_portfolio_value(current_price)}")
    print(f"{interval_name} - EMAV Agent Portfolio Value: {emav_agent.get_portfolio_value(current_price)}")
    return df

# Define intervals in seconds
intervals = {
    
    "1 hour": 3600,
    
}

# Track the last run time for each interval
last_run_times = {name: datetime.now() for name in intervals}

# Main loop to update data and agents
while True:
    current_time = datetime.now()
    
    for interval_name, interval_seconds in intervals.items():
        if (current_time - last_run_times[interval_name]).total_seconds() >= interval_seconds:
            print(f"Running update for interval: {interval_name}")
            df = update_agents(df, current_time, interval_name)
            last_run_times[interval_name] = current_time
    
    time.sleep(60)  # Check every minute


Running update for interval: 1 hour
2024-07-23 16:56:15.191888: MAC Hold
2024-07-23 16:56:15.192888: Mean Reversion Hold
2024-07-23 16:56:15.193893: Momentum Hold
2024-07-23 16:56:15.194892: MACD Hold
2024-07-23 16:56:15.195892: Bollinger Bands Hold
2024-07-23 16:56:15.196893: SMA-RSI Hold
2024-07-23 16:56:15.198487: Composite Agent Hold
2024-07-23 16:56:15.198487: EMA-Volume Hold
1 hour - MAC Agent Portfolio Value: 100000.0
1 hour - Dummy Agent Portfolio Value: 100000.0
1 hour - MeanReversion Agent Portfolio Value: 100000.0
1 hour - Momentum Agent Portfolio Value: 100000.0
1 hour - MACD Agent Portfolio Value: 100000.0
1 hour - BollingerBands Agent Portfolio Value: 100000.0
1 hour - RSI Agent Portfolio Value: 100000.0
1 hour - Composite Agent Portfolio Value: 100000.0
1 hour - EMAV Agent Portfolio Value: 100000.0


C:\Users\Pawat_Pai\AppData\Local\Temp\ipykernel_37104\3663338114.py:7: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df = pd.concat([df, new_data])
C:\Users\Pawat_Pai\AppData\Local\Temp\ipykernel_37104\3663338114.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data])


Running update for interval: 1 hour
2024-07-23 17:56:15.364306: MAC Hold
2024-07-23 17:56:15.364306: Dummy Buy at 66905.99
2024-07-23 17:56:15.365306: Mean Reversion Hold
2024-07-23 17:56:15.365306: Momentum Hold
2024-07-23 17:56:15.368378: Bollinger Bands Hold
2024-07-23 17:56:15.368882: SMA-RSI Buy at 66905.99
2024-07-23 17:56:15.369887: Composite Agent Hold
2024-07-23 17:56:15.369887: EMA-Volume Hold
1 hour - MAC Agent Portfolio Value: 100000.0
1 hour - Dummy Agent Portfolio Value: 100000.0
1 hour - MeanReversion Agent Portfolio Value: 100000.0
1 hour - Momentum Agent Portfolio Value: 100000.0
1 hour - MACD Agent Portfolio Value: 100000.0
1 hour - BollingerBands Agent Portfolio Value: 100000.0
1 hour - RSI Agent Portfolio Value: 100000.0
1 hour - Composite Agent Portfolio Value: 100000.0
1 hour - EMAV Agent Portfolio Value: 100000.0
Running update for interval: 1 hour
2024-07-23 18:56:15.521723: MAC Hold
2024-07-23 18:56:15.521723: Dummy Hold
2024-07-23 18:56:15.523043: Mean Reversi

KeyboardInterrupt: 